# 1. Import the necessary libraries for you to be able to query 4square API

In [5]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
from dotenv import load_dotenv

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [7]:
load_dotenv()

url = "https://api.foursquare.com/v3/places/search"

params = {
    "query": "cafe",
    "ll": "41.39,2.18",
    "radius": 500,
    "limit": 1
}

headers = {
    "Authorization": os.getenv("passwd"),
    "accept": "application/json"
}

response = requests.get(url, params=params, headers=headers)

# Check the status code to see if the request was successful
if response.status_code == 200:
    # If successful, the response JSON will be in the `response.json()` dictionary
    result = response.json()
    print(result)
else:
    # If the request was unsuccessful, print the status code and text
    print(f"Request failed with status code {response.status_code}: {response.text}")

{'results': [{'fsq_id': '5a1e8e634382ab7bf85940f3', 'categories': [{'id': 13034, 'name': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}}], 'chains': [], 'distance': 249, 'geocodes': {'main': {'latitude': 41.388319, 'longitude': 2.177882}, 'roof': {'latitude': 41.388319, 'longitude': 2.177882}}, 'link': '/v3/places/5a1e8e634382ab7bf85940f3', 'location': {'address': "Calle D'en Mònec, 6", 'admin_region': 'Cataluña', 'country': 'ES', 'cross_street': '', 'formatted_address': "Calle D'en Mònec, 6, 08003 Barcelona Catalonia", 'locality': 'Barcelona', 'postcode': '08003', 'region': 'Catalonia'}, 'name': 'La Masala Cafe', 'related_places': {}, 'timezone': 'Europe/Madrid'}], 'context': {'geo_bounds': {'circle': {'center': {'latitude': 41.39, 'longitude': 2.18}, 'radius': 500}}}}


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [10]:
# your code here
import requests
import os
from dotenv import load_dotenv

load_dotenv()

url = "https://api.foursquare.com/v3/places/search"

params = {
    "query": "cafe",
    "ll": "41.39,2.18",
    "radius": 250,
    "limit": 5
}

headers = {
    "Authorization": os.getenv("passwd"),
    "accept": "application/json"
}

response = requests.get(url, params=params, headers=headers)

# Check the status code to see if the request was successful
if response.status_code == 200:
    # If successful, the response JSON will be in the `response.json()` dictionary
    result = response.json()
    print(result)
else:
    # If the request was unsuccessful, print the status code and text
    print(f"Request failed with status code {response.status_code}: {response.text}")


{'results': [{'fsq_id': '4bb9f03653649c74b46748fb', 'categories': [{'id': 13034, 'name': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}}, {'id': 13035, 'name': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}, {'id': 13345, 'name': 'Spanish Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_', 'suffix': '.png'}}], 'chains': [], 'distance': 134, 'geocodes': {'main': {'latitude': 41.389473, 'longitude': 2.181254}, 'roof': {'latitude': 41.389473, 'longitude': 2.181254}}, 'link': '/v3/places/4bb9f03653649c74b46748fb', 'location': {'address': 'Calle del Comerç, 4', 'admin_region': 'Cataluña', 'country': 'ES', 'formatted_address': 'Calle del Comerç, 4, 08003 Barcelona Catalonia', 'locality': 'Barcelona', 'postcode': '08003', 'region': 'Catalonia'}, 'name': 'El Montana Cafè & Restaurant', 'related_places': {}, 'timezone': 'Europe/Madrid'}, {'fsq_i

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [17]:
load_dotenv()

def search_places(coordinates, query, limit):
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": query,
        "ll": coordinates,
        "radius": 5000,
        "limit": limit
    }

    headers = {
        "Authorization": os.getenv("passwd"),
        "accept": "application/json"
    }

    response = requests.get(url, params=params, headers=headers)

    # Check the status code to see if the request was successful
    if response.status_code == 200:
        # If successful, the response JSON will be in the `response.json()` dictionary
        result = response.json()
        return result
    else:
        # If the request was unsuccessful, return the status code and text
        return {
            "error": {
                "status_code": response.status_code,
                "message": response.text
            }
        }

In [18]:
result = search_places("35.89,14.43", "McDonald's", 5)


In [19]:
result

{'results': [{'fsq_id': '5352ac34498ed4c1fd3f1785',
   'categories': [{'id': 13145,
     'name': 'Fast Food Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 1837,
   'geocodes': {'main': {'latitude': 35.906162, 'longitude': 14.42568}},
   'link': '/v3/places/5352ac34498ed4c1fd3f1785',
   'location': {'address': 'Naxxar Road',
    'country': 'MT',
    'cross_street': '',
    'formatted_address': 'Naxxar Road, Gharghur',
    'locality': 'Gharghur'},
   'name': "McDonald's",
   'related_places': {},
   'timezone': 'Europe/Malta'},
  {'fsq_id': '533ef5c1498e1eb6bc9bef2b',
   'categories': [{'id': 13145,
     'name': 'Fast Food Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 3759,
   'geocodes': {'main': {'latitude': 35.916992, 'longitude': 14.455188}},
   'link': '/v3/places/533ef5c14

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [28]:
# your code here
load_dotenv()

def search_places(coordinates, query, limit):
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": query,
        "ll": coordinates,
        "radius": 25000,
        "limit": limit
    }

    headers = {
        "Authorization": os.getenv("passwd"),
        "accept": "application/json"
    }

    response = requests.get(url, params=params, headers=headers)

    # Check the status code to see if the request was successful
    if response.status_code == 200:
        # If successful, the response JSON will be in the `response.json()` dictionary
        result = response.json()
        filename = f"data/{query.replace(' ', '_')}.json"
        with open(filename, "w") as file:
            json.dump(result, file)
        return result
    else:
        # If the request was unsuccessful, return the status code and text
        return {
            "error": {
                "status_code": response.status_code,
                "message": response.text
            }
        }

# Example usage:
result = search_places("35.89,14.43", "McDonald's", 5)
print(result)


{'results': [{'fsq_id': '4d0e497a7d28721e9070fd20', 'categories': [{'id': 13145, 'name': 'Fast Food Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}}], 'chains': [], 'distance': 7172, 'geocodes': {'main': {'latitude': 35.851613, 'longitude': 14.494035}}, 'link': '/v3/places/4d0e497a7d28721e9070fd20', 'location': {'address': 'Aviation St.', 'country': 'MT', 'cross_street': 'MLA Airport', 'formatted_address': 'Aviation St. (MLA Airport), Kirkop', 'locality': 'Kirkop', 'region': 'Kirkop'}, 'name': "McDonald's", 'related_places': {}, 'timezone': 'Europe/Malta'}, {'fsq_id': '533ef5c1498e1eb6bc9bef2b', 'categories': [{'id': 13145, 'name': 'Fast Food Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}}], 'chains': [], 'distance': 3759, 'geocodes': {'main': {'latitude': 35.916992, 'longitude': 14.455188}}, 'link': '/v3/places/533ef5c1498e1eb6bc9bef2b', 'location': {'country': 'MT', 

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [24]:
# your code here
import pymongo


In [27]:
load_dotenv()

def search_places(coordinates, query, limit):
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": query,
        "ll": coordinates,
        "radius": 500,
        "limit": limit
    }

    headers = {
        "Authorization": os.getenv("passwd"),
        "accept": "application/json"
    }

    response = requests.get(url, params=params, headers=headers)

    # Check the status code to see if the request was successful
    if response.status_code == 200:
        # If successful, the response JSON will be in the `response.json()` dictionary
        result = response.json()
        filename = f"data/{query.replace(' ', '_')}.json"
        with open(filename, "w") as file:
            json.dump(result, file)

        client = pymongo.MongoClient(os.getenv("MONGO_URI"))
        db = client["foursquare"]
        collection = db["ironcoffee"]
        collection.insert_many(result["places"]["items"])
        return result
    else:
        # If the request was unsuccessful, return the status code and text
        return {
            "error": {
                "status_code": response.status_code,
                "message": response.text
            }
        }

# Example usage:
result = search_places("35.89,14.43", "McDonald's", 5)
print(result)

KeyError: 'places'